In [ ]:
# Examples related to gridding, last updated 11/3/2021, Kluster 0.8.2
# uses the multibeam file hstb_kluster/test_data/0009_20170523_181119_FA2806.all
# Written by Eric Younkin

# gridding is the process of taking point data (x y and z) and grouping/binning the data within cells of a predetermined
# size (resolution).  Kluster uses the bathygrid module (https://github.com/noaa-ocs-hydrography/bathygrid) to do this
# gridding.  Points are split into tiles, and those tiles contain the points/grid data.  Grids can be single resolution,
# where all tiles contain grids of the same size, or a tile can be variable resolution, where each tile can be its own
# resolution.

In [ ]:
from HSTB.kluster.fqpr_convenience import reload_data, generate_new_surface, update_surface, reload_surface
from HSTB.kluster.fqpr_intelligence import intel_process

In [ ]:
# we start with one of the preferred processing steps from the data_processing example
_, fq = intel_process(r"C:\data_dir\0009_20170523_181119_FA2806.all")
fq = fq[0]  # this should just be a list of one element if you have just one sonar/day of data, take the first one to get the data

In [ ]:
# or we can just reload if you have data from before
fq = reload_data(r"C:\data_dir\em2040_40111_05_23_2017")

In [ ]:
# from here, we simply generate a new grid.  Let's start with a simple one, a single resolution grid (where the cells
#  are all the same size/resolution) and that resolution is automatically determined by the depth of the data.

# the points from the Fqpr Kluster object will be added to the grid object, and the grid object will automatically run
#  the gridding routine with the 'mean' algorithm

# the one setting we will use is the output_path, where we specify the output directory.  If we were not to do this,
#  the grid will exist entirely in memory, which might cause memory issues later
bs = generate_new_surface(fq, output_path=r"C:\data_dir\mygridpath")

In [ ]:
# adding points and gridding will set some useful attributes in the class
bs.output_folder

In [ ]:
bs.min_time  # this is based on the data time, not the time it was added

In [ ]:
bs.max_time

In [ ]:
bs.min_x

In [ ]:
bs.max_x

In [ ]:
bs.min_y

In [ ]:
bs.max_y

In [ ]:
bs.resolutions  # this grid is a single resolution 8 meter cell grid

In [ ]:
# as mentioned before, data is organized first in tiles.  These tiles correspond to the folders in the grid directory.
# here we have two total tiles that represent the grid.  Each is 1024 meters square.
bs.tile_size

In [ ]:
bs.tiles

In [ ]:
# inside each of the tiles should be the grid and points that exist within the tile boundary
tile = bs.tiles[0][0]

In [ ]:
# points
tile.data

In [ ]:
# grid data
tile.cells

In [ ]:
# depth gridded data for the 8.0 meter resolution grid, can see that it is mostly empty space in this case
tile.cells[8.0]['depth'].compute()

In [ ]:
# since we chose resolution=None, and auto_resolution_mode=depth (the default arguments in generate_new_surface), the resolution
#  for the tile cells will be created using the depth resolution lookup and the tile mean depth.  Next category after 90 meters is
#  the 160 meter depth range, which corresponds to 8 meter resolution.

from bathygrid.bgrid import depth_resolution_lookup

In [ ]:
tile.mean_depth

In [ ]:
depth_resolution_lookup[80]

In [ ]:
depth_resolution_lookup[160]

In [ ]:
# now we go back to the grid to look at data management.  When we added points to it from the fq object, they were registered
#  under a tag in the grid object container.  The grid retains the file that the points came from as well as the date they were
#  added.

# you'll see that the data key is 'em2040_40111_05_23_2017_0' with the _0 at the end.  Since there can be millions of points
#  in one Fqpr object, we sometimes have to add in chunks.  If that were the case, you would see 'em2040_40111_05_23_2017_1',
#  'em2040_40111_05_23_2017_2', etc.

bs.container

In [ ]:
bs.container_timestamp

In [ ]:
# The tile is the object that actually holds the points, so we can look at the tile container to figure out which points are
#  from which container.  Here we can see that all the points from index 0 to index 48322 belong to this container.
tile.container

In [ ]:
cont_data = tile.data[0:48322]

In [ ]:
# this allows us to easily add/remove points and track where they come from.  If we update the points in Kluster by
#  re soundvelocity correcting or something and we need to then update the points in the grid, we can just remove all the points
#  associated with that Fqpr object and re-add them.  This is what update_surface does.

# lets show how to update the points by adding/removing this fqpr object
bs = update_surface(bs, add_fqpr=fq, remove_fqpr=fq)

In [ ]:
# so we can see that the points were removed, added, and the tiles were then gridded.  It is kind of brute-force, non
#  elegant, but it is pretty reliable.

# there are other routines you can do with update_surface.  You can also do a full regrid of the data if you like, although
#  this really shouldn't ever be necessary.  Ideally you should be able to update the grid (regrid_option=update) while adding
#  removing points as we did earlier
bs = update_surface(bs, regrid_option='full')

In [ ]:
# so you can see that the grid was updated without adding or removing points.

# finally let's talk about exporting the grid.  bathygrid allows you to export to GDAL formats, although only GeoTIFF and
#  BAG are supported currently, as well as a more basic csv option.  GeoTIFF is the simplest, so let's start there.

bs.export(r'C:\data_dir\mygridpath\mytiff.tif', export_format='geotiff')

In [ ]:
# you'll see a 'mytiff_8.0_1.tif' file in there, implying that it is a 8.0 meter tif and is the 1st of the series.  Bathygrid
#  will export in tiles, so you might get a 'mytiff_8.0_2.tif', etc. as well.  This prevents the exported grids from being
#  too large to interact with efficiently.

# if you were to generate a grid in variable resolution mode, you would get one tif per tile, per resolution.  Bathygrid
#  does not currently export using multipage tifs or variable resolution BAGs.

# you can do the same thing with BAG, although BAG is a little more metadata heavy.  You can just leave the defaults or you
# can populate some of the keyword arguments.  Check the docs to learn more about the possible options.
bs.export(r'C:\data_dir\mygridpath\mybag.bag', organizational_name='noaa', position_name='scientist?', export_format='bag')